# US hit by 1-in-1,000-year flood

James Munroe, jmunroe@2i2c.org

In [37]:
import os
import shutil
import fsspec
import ujson
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import dask
import hvplot.xarray

## From the evening news

I was listening to the news tonight and learned that Dallas, TX is currently experiencing significant flooding.  For example, the Washington Post reports:

https://www.washingtonpost.com/nation/2022/08/22/dallas-texas-flash-floods/

> In some isolated areas, the rainfall totals would be considered a 1-in-1,000-year flood — a remarkable reversal given the dramatic drought that Dallas had faced for months. Several rainfall gauges recorded more than 10 inches. A record-breaking 3.01 inches of rain was also recorded in one hour at Dallas-Fort Worth International Airport.

> The downpour marked the latest such flood in the past few weeks across the United States. In one week alone, three 1-in-1,000-year rain events occurred, inundating St. Louis, eastern Kentucky and southeastern Illinois. The term, often considered controversial in part because it’s misunderstood, is used to describe a rainfall event that is expected once every 1,000 years, meaning it has just a 0.1 percent chance of happening in any given year — but such events can occur much more frequently.

> ...

> One rain gauge in Harris County, Tex., tallied more than 14.9 inches of rain within just a 12-hour period, more than 40 percent of the area’s yearly rainfall, according to Jeff Lindner, a meteorologist for the county. Such rates of precipitation are nearly impossible for soils — not to mention impervious paved surfaces — to absorb without runoff that can cause flash flooding.

## National Water Model and the AWI-CIROH JupyterHub

Considering that the AWI-CIROH now has a 2i2c managed JupyterHub running on Google Compute Platform (GCP) and a signifcant amount of [National Water Model](https://water.noaa.gov/about/nwm) data has already been made available on a bucket, I will explore this dataset by looking that some of the historical data for regions that have experienced intense rainfall and flooding recently.

Hourly data is available from 2018-09-17 to 2022-08-22 and growing every day.

A 2020 [blog post](https://medium.com/pangeo/cloud-performant-netcdf4-hdf5-with-zarr-fsspec-and-intake-3d3a3e7cb935) on *Cloud-Performant NetCDF4/HDF5 with Zarr, Fsspec, and Intake* by Rich Signell (USGS), Martin Durant (Anaconda) and Aleksandar Jelenak (HDF Group) demonstrated how to read data from the NWM on Amazon Web Services. Let's see if we can make this work with this data on GCP.

In [2]:
import gcsfs

In [3]:
fs = gcsfs.GCSFileSystem()

### Opening a single NetCDF-4 file

In [4]:
best_hour = 'f001'
var = 'land'

In [25]:
fname = f'national-water-model/nwm.20220801/short_range/nwm.t00z.short_range.{var}.{best_hour}.conus.nc'

ds = xr.open_dataset(fs.open(fname))
soilsat = ds.SOILSAT_TOP.load()

In [27]:
soilsat.hvplot.quadmesh('x', 'y', rasterize=True)

:DynamicMap   [time]
   :Image   [x,y]   (SOILSAT_TOP)

Make a list of all hours for one day of August 2022.

In [55]:
flist = []
for day in range(1,2):
    for i in range(24):
        flist.append(f'national-water-model/nwm.2022080{day}/short_range/nwm.t{i:02d}z.short_range.{var}.{best_hour}.conus.nc')

In [58]:
fs2 = fsspec.filesystem('')

In [59]:
json_dir = 'jsons/'

In [60]:
try:
    shutil.rmtree(json_dir)
except:
    pass
finally:
    os.makedirs(json_dir)

In [61]:
so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first') 

In [62]:
def gen_json(u):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, flist[0], inline_threshold=300)
        p = u.split('/')
        date = p[1]
        fname = p[3]
        outf = f'{json_dir}{date}.{fname}.json'
        print(outf)
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [63]:
%%time
dask.compute(*[dask.delayed(gen_json)(u) for u in flist], retries=10);

jsons/nwm.20220801.nwm.t15z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t03z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t16z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t22z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t18z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t19z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t10z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t11z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t02z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t13z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t06z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t00z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t04z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t05z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t23z.short_range.land.f001.conus.nc.json
jsons/nwm.20220801.nwm.t14z.short_range.

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

### Try to open one of these files

In [85]:
ds = xr.open_dataset("reference://", 
                     engine="zarr",
                     backend_kwargs={
                          "consolidated": False,
                          "storage_options": {"fo": "jsons/nwm.20220801.nwm.t00z.short_range.land.f001.conus.nc.json",
                                          }
                     })

But all the values and coordinates are "nan". Are there additional options I need to be passing to `SingleHdf5ToZarr` or `xr.open_dataset` ?

In [85]:
ds

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'ACCET' has multiple fill values {0, -999900}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'FSNO' has multiple fill values {-9999000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'SNEQV' has multiple fill values {0, -99990}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'SNOWH' has multiple fill values {-99990000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 

<xarray.Dataset>
Dimensions:         (time: 1, y: 3840, x: 4608, reference_time: 1)
Coordinates:
  * reference_time  (reference_time) datetime64[ns] 2022-08-01
  * time            (time) datetime64[ns] 2022-08-01T01:00:00
  * x               (x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
  * y               (y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    ACCET           (time, y, x) float64 ...
    FSNO            (time, y, x) float64 ...
    SNEQV           (time, y, x) float64 ...
    SNOWH           (time, y, x) float64 ...
    SNOWT_AVG       (time, y, x) float64 ...
    SOILSAT_TOP     (time, y, x) float64 ...
    crs             object ...
Attributes:
    Conventions:                CF-1.6
    GDAL_DataType:              Generic
    NWM_version_number:         v2.2
    TITLE:                      OUTPUT FROM NWM v2.2
    code_version:               v5.2.0-beta2
    model_configuration:        short_range
    model_initialization_time:  2022-08-01_00:00:00
    model_output_type:          land
    model_output_valid_time:    2022-08-01_01:00:00
    model_total_valid_times:    18
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...

In [87]:
ds.SOILSAT_TOP.load()

<xarray.DataArray 'SOILSAT_TOP' (time: 1, y: 3840, x: 4608)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2022-08-01T01:00:00
  * x        (x) float64 nan nan nan nan nan nan nan ... nan nan nan nan nan nan
  * y        (y) float64 nan nan nan nan nan nan nan ... nan nan nan nan nan nan
Attributes:
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       fraction of soil saturation, top 2 layers
    units:           1
    valid_range:     [0, 1000]

### Combine all the JSONs into a single reference JSON

In [77]:
flist2 = fs2.glob(f'{json_dir}/*.json')
furls = sorted(flist2)

In [78]:
len(furls)

24

In [79]:
mzz = MultiZarrToZarr(furls,
                     concat_dims=['time'],
                          target_options={
        'decode_cf' : False,
        'mask_and_scale' : False,
        'decode_times' : False,
        'use_cftime' : False,
        'drop_variables': ['reference_time', 'crs'],
        'decode_coords' : False
    })

In [68]:
%%time
mzz.translate('nwm.json')

CPU times: user 129 ms, sys: 8.97 ms, total: 138 ms
Wall time: 167 ms


In [83]:
ds = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": 'nwm.json'
        },
        "consolidated": False
    }
)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'ACCET' has multiple fill values {0, -999900}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'FSNO' has multiple fill values {-9999000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'SNEQV' has multiple fill values {0, -99990}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'SNOWH' has multiple fill values {-99990000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 

In [84]:
print(ds.SOILSAT_TOP)

<xarray.DataArray 'SOILSAT_TOP' (time: 24, y: 3840, x: 4608)>
[424673280 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2022-08-01T01:00:00 ... 2022-08-02
  * x        (x) float64 nan nan nan nan nan nan nan ... nan nan nan nan nan nan
  * y        (y) float64 nan nan nan nan nan nan nan ... nan nan nan nan nan nan
Attributes:
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       fraction of soil saturation, top 2 layers
    units:           1
    valid_range:     [0, 1000]
